In [1]:
import stanza
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import tqdm as notebook_tqdm

In [2]:
nlp = stanza.Pipeline(lang="en",use_gpu=True)

2024-01-04 09:51:22 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-01-04 09:51:24 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2024-01-04 09:51:24 WARNING: GPU requested, but is not available!
2024-01-04 09:51:24 INFO: Using device: cpu
2024-01-04 09:51:24 INFO: Loading: tokenize
2024-01-04 09:51:25 INFO: Loading: pos
2024-01-04 09:51:25 INFO: Loading: lemma
2024-01-04 09:51:25 INFO: Loading: constituency
2024-01-04 09:51:25 INFO: Loading: depparse
2024-01-04 09:51:26 INFO: Loading: sentiment
2024-01-04 09:51:26 INFO: Loading: ner
2024-01-04 09:51:26 INFO: Done loading processors!


In [3]:
doc = nlp("An elderly man sitting in a small shop")
print("token:")
doc.sentences[0].print_tokens()

token:
<Token id=1;words=[<Word id=1;text=An;lemma=a;upos=DET;xpos=DT;feats=Definite=Ind|PronType=Art;head=3;deprel=det>]>
<Token id=2;words=[<Word id=2;text=elderly;lemma=elderly;upos=ADJ;xpos=JJ;feats=Degree=Pos;head=3;deprel=amod>]>
<Token id=3;words=[<Word id=3;text=man;lemma=man;upos=NOUN;xpos=NN;feats=Number=Sing;head=0;deprel=root>]>
<Token id=4;words=[<Word id=4;text=sitting;lemma=sit;upos=VERB;xpos=VBG;feats=VerbForm=Ger;head=3;deprel=acl>]>
<Token id=5;words=[<Word id=5;text=in;lemma=in;upos=ADP;xpos=IN;head=8;deprel=case>]>
<Token id=6;words=[<Word id=6;text=a;lemma=a;upos=DET;xpos=DT;feats=Definite=Ind|PronType=Art;head=8;deprel=det>]>
<Token id=7;words=[<Word id=7;text=small;lemma=small;upos=ADJ;xpos=JJ;feats=Degree=Pos;head=8;deprel=amod>]>
<Token id=8;words=[<Word id=8;text=shop;lemma=shop;upos=NOUN;xpos=NN;feats=Number=Sing;head=4;deprel=obl>]>


In [4]:
# Import dataframe
import pandas as pd
test_file = 'D:\\Suwija Putra\\datasetTextualEntailment\\snli_1.0\\snli_1.0_test.jsonl'
df_test = pd.read_json(test_file, lines=True) 
df_test

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[neutral, contradiction, contradiction, neutra...",2677109430.jpg#1,neutral,2677109430.jpg#1r1n,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,The church has cracks in the ceiling.,( ( The church ) ( ( has ( cracks ( in ( the c...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ ha...
1,"[entailment, entailment, entailment, neutral, ...",2677109430.jpg#1,entailment,2677109430.jpg#1r1e,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,The church is filled with song.,( ( The church ) ( ( is ( filled ( with song )...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ is...
2,"[contradiction, contradiction, contradiction, ...",2677109430.jpg#1,contradiction,2677109430.jpg#1r1c,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,A choir singing at a baseball game.,( ( ( A choir ) ( singing ( at ( a ( baseball ...,(ROOT (NP (NP (DT A) (NN choir)) (VP (VBG sing...
3,"[neutral, neutral, neutral, neutral, neutral]",6160193920.jpg#4,neutral,6160193920.jpg#4r1n,"A woman with a green headscarf, blue shirt and...",( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,The woman is young.,( ( The woman ) ( ( is young ) . ) ),(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...
4,"[entailment, entailment, contradiction, entail...",6160193920.jpg#4,entailment,6160193920.jpg#4r1e,"A woman with a green headscarf, blue shirt and...",( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,The woman is very happy.,( ( The woman ) ( ( is ( very happy ) ) . ) ),(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...
...,...,...,...,...,...,...,...,...,...,...
9995,"[contradiction, contradiction, contradiction, ...",4378810163.jpg#4,contradiction,4378810163.jpg#4r1c,Two women are observing something together.,( ( Two women ) ( ( are ( ( observing somethin...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,Two women are standing with their eyes closed.,( ( ( Two women ) ( are ( standing ( with ( th...,(ROOT (S (NP (NP (CD Two) (NNS women)) (SBAR (...
9996,"[entailment, entailment, entailment, contradic...",4378810163.jpg#4,entailment,4378810163.jpg#4r1e,Two women are observing something together.,( ( Two women ) ( ( are ( ( observing somethin...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,Two girls are looking at something.,( ( Two girls ) ( ( are ( looking ( at somethi...,(ROOT (S (NP (CD Two) (NNS girls)) (VP (VBP ar...
9997,"[contradiction, contradiction, contradiction, ...",152881593.jpg#1,contradiction,152881593.jpg#1r1c,A man in a black leather jacket and a book in ...,( ( ( ( ( A man ) ( in ( a ( black ( leather j...,(ROOT (S (NP (NP (NP (DT A) (NN man)) (PP (IN ...,A man is flying a kite.,( ( A man ) ( ( is ( flying ( a kite ) ) ) . ) ),(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...
9998,"[entailment, entailment, entailment, neutral, ...",152881593.jpg#1,entailment,152881593.jpg#1r1e,A man in a black leather jacket and a book in ...,( ( ( ( ( A man ) ( in ( a ( black ( leather j...,(ROOT (S (NP (NP (NP (DT A) (NN man)) (PP (IN ...,A man is speaking in a classroom.,( ( A man ) ( ( is ( speaking ( in ( a classro...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...


In [5]:
df_test.shape

(10000, 10)

In [6]:
import string
df_test_kalimat1 = df_test
df_test_kalimat1 = df_test_kalimat1['sentence1']
# Ambil 10 data pertama
df_test_kalimat2 = df_test
# Ambil kalimat kedua saja sebagai contoh
df_test_kalimat2 = df_test_kalimat2['sentence2']

df_label = df_test
df_label = df_label['gold_label']
# Ubah ke lowercase
def remove_punctuation(text):
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
df_train_kalimat1 = df_test_kalimat1.apply(remove_punctuation)
df_train_kalimat2 = df_test_kalimat2.apply(remove_punctuation)

In [7]:
list_dependencies_2 = list()
for i in df_test_kalimat2:
    # print(i)
    doc = nlp(i)
    dependencies_list = doc.sentences[0].dependencies
    list_dependencies = list()
    for x in range(len(dependencies_list)):
        # print(dependencies_list[x][0].text)
        # print(dependencies_list[x][2].text)
        list_dependencies.append([dependencies_list[x][0].text,dependencies_list[x][2].text])
    # print(list_dependencies)
    list_dependencies_2.append(list_dependencies)
# list_dependencies_all

list_dependencies_1 = list()
for i in df_test_kalimat1:
    # print(i)
    doc = nlp(i)
    dependencies_list = doc.sentences[0].dependencies
    list_dependencies = list()
    for x in range(len(dependencies_list)):
        # print(dependencies_list[x][0].text)
        # print(dependencies_list[x][2].text)
        list_dependencies.append([dependencies_list[x][0].text,dependencies_list[x][2].text])
    # print(list_dependencies)
    list_dependencies_1.append(list_dependencies)

In [8]:
filtered_list1 = [[sublist for sublist in subsublist if 'ROOT' not in sublist] for subsublist in list_dependencies_1]
filtered_list2 = [[sublist for sublist in subsublist if 'ROOT' not in sublist] for subsublist in list_dependencies_2]
print(df_test_kalimat1[0])
print(filtered_list1[0])

This church choir sings to the masses as they sing joyous songs from the book at a church.
[['choir', 'This'], ['choir', 'church'], ['sings', 'choir'], ['masses', 'to'], ['masses', 'the'], ['sings', 'masses'], ['sing', 'as'], ['sing', 'they'], ['sings', 'sing'], ['songs', 'joyous'], ['sing', 'songs'], ['book', 'from'], ['book', 'the'], ['sing', 'book'], ['church', 'at'], ['church', 'a'], ['book', 'church'], ['sings', '.']]


In [9]:
# Head
head_1 = list()
for item in filtered_list1:
    head_item = list()
    for i in item:
        head_item.append(i[0])
    head_1.append(head_item)
# print(head_1)

dependecy_1 = list()
for item in filtered_list1:
    dependecy_item = list()
    for i in item:
        dependecy_item.append(i[1])
    dependecy_1.append(dependecy_item)
# print(dependecy_1)

head_2 = list()
for item in filtered_list2:
    head_item = list()
    for i in item:
        head_item.append(i[0])
    head_2.append(head_item)
# print(head_1)

dependecy_2 = list()
for item in filtered_list2:
    dependecy_item = list()
    for i in item:
        dependecy_item.append(i[1])
    dependecy_2.append(dependecy_item)

In [10]:
head_dependecy = pd.DataFrame({'head1':head_1,'dependecy1':dependecy_1,'head2':head_2,'dependecy2':dependecy_2})
head_dependecy = pd.concat([head_dependecy,df_label],axis=1)
head_dependecy

,head1,dependecy1,head2,dependecy2,gold_label
0,"[choir, choir, sings, masses, masses, sings, s...","[This, church, choir, to, the, masses, as, the...","[church, has, has, ceiling, ceiling, has, has]","[The, church, cracks, in, the, ceiling, .]",neutral
1,"[choir, choir, sings, masses, masses, sings, s...","[This, church, choir, to, the, masses, as, the...","[church, filled, filled, song, filled, filled]","[The, church, is, with, song, .]",entailment
2,"[choir, choir, sings, masses, masses, sings, s...","[This, church, choir, to, the, masses, as, the...","[choir, choir, game, game, game, singing, choir]","[A, singing, at, a, baseball, game, .]",contradiction
3,"[woman, headscarf, headscarf, headscarf, woman...","[A, with, a, green, headscarf, ,, blue, shirt,...","[woman, young, young, young]","[The, woman, is, .]",neutral
4,"[woman, headscarf, headscarf, headscarf, woman...","[A, with, a, green, headscarf, ,, blue, shirt,...","[woman, happy, happy, happy, happy]","[The, woman, is, very, .]",entailment
...,...,...,...,...,...
9995,"[women, observing, observing, observing, obser...","[Two, women, are, something, together, .]","[women, standing, standing, closed, eyes, clos...","[Two, women, are, with, their, eyes, closed, .]",contradiction
9996,"[women, observing, observing, observing, obser...","[Two, women, are, something, together, .]","[girls, looking, looking, something, looking, ...","[Two, girls, are, at, something, .]",entailment
9997,"[man, speaks, jacket, jacket, jacket, jacket, ...","[A, man, in, a, black, leather, jacket, and, a...","[man, flying, flying, kite, flying, flying]","[A, man, is, a, kite, .]",contradiction
9998,"[man, speaks, jacket, jacket, jacket, jacket, ...","[A, man, in, a, black, leather, jacket, and, a...","[man, speaking, speaking, classroom, classroom...","[A, man, is, in, a, classroom, .]",entailment


In [11]:
head_dependecy['head1'] = head_dependecy['head1'].apply(lambda x: ' '.join(x))
head_dependecy['dependecy1'] = head_dependecy['dependecy1'].apply(lambda x: ' '.join(x))
head_dependecy['head2'] = head_dependecy['head2'].apply(lambda x: ' '.join(x))
head_dependecy['dependecy2'] = head_dependecy['dependecy2'].apply(lambda x: ' '.join(x))
pd.set_option('display.max_colwidth',1)
head_dependecy

,head1,dependecy1,head2,dependecy2,gold_label
0,choir choir sings masses masses sings sing sing sings songs sing book book sing church church book sings,This church choir to the masses as they sing joyous songs from the book at a church .,church has has ceiling ceiling has has,The church cracks in the ceiling .,neutral
1,choir choir sings masses masses sings sing sing sings songs sing book book sing church church book sings,This church choir to the masses as they sing joyous songs from the book at a church .,church filled filled song filled filled,The church is with song .,entailment
2,choir choir sings masses masses sings sing sing sings songs sing book book sing church church book sings,This church choir to the masses as they sing joyous songs from the book at a church .,choir choir game game game singing choir,A singing at a baseball game .,contradiction
3,woman headscarf headscarf headscarf woman shirt shirt headscarf grin grin big grin headscarf woman,"A with a green headscarf , blue shirt and a very big grin .",woman young young young,The woman is .,neutral
4,woman headscarf headscarf headscarf woman shirt shirt headscarf grin grin big grin headscarf woman,"A with a green headscarf , blue shirt and a very big grin .",woman happy happy happy happy,The woman is very .,entailment
...,...,...,...,...,...
9995,women observing observing observing observing observing,Two women are something together .,women standing standing closed eyes closed standing standing,Two women are with their eyes closed .,contradiction
9996,women observing observing observing observing observing,Two women are something together .,girls looking looking something looking looking,Two girls are at something .,entailment
9997,man speaks jacket jacket jacket jacket man book book man hand hand book classroom classroom speaks speaks,A man in a black leather jacket and a book in his hand in a classroom .,man flying flying kite flying flying,A man is a kite .,contradiction
9998,man speaks jacket jacket jacket jacket man book book man hand hand book classroom classroom speaks speaks,A man in a black leather jacket and a book in his hand in a classroom .,man speaking speaking classroom classroom speaking speaking,A man is in a classroom .,entailment


In [12]:
# save the file
head_dependecy.to_csv('D:\\Suwija Putra\\datasetTextualEntailment\\indonli\\dependency_test_snli.csv', index=False) # change the name of the file